In [1]:
#This one uses an active scenario in the GUI


from veneer.manage import start, create_command_line, kill_all_now
import veneer
import pandas as pd
import gc
import numpy as np
import os
#from veneer.pest_runtime import *


veneer_port = 9876
v = veneer.Veneer(veneer_port)



#nodeOfInterestName = '120212A'
#outputCSV = 'D:/aa/testingSubCats.csv'
#outputCSV = r'C:\DDrive\WetTropics\Johnstone\contribCatchs.csv'

In [2]:
#Check we have the right scenario
sDate = v.model.get('scenario.CurrentConfiguration.StartDate')
print(sDate)

eDate = v.model.get('scenario.CurrentConfiguration.EndDate')
print(eDate)

#v.run_model()

1/01/1970 12:00:00 AM
30/06/2021 12:00:00 AM


In [3]:
the_network = v.network()
#outlets = the_network.outlet_nodes()
#outlets

In [4]:
def processLinks(theNetwork, currentLink):
    theLinkName = currentLink['properties']['name']
    theLinkID = currentLink['id']
    
    #print("Now processing: " + theLinkName)
          
    # I think this will actually return a catchment record, not just an ID
    theCat = theNetwork['features'].find_by_link(theLinkID)[0]
    
    #print(theCat)
    
    thisCatName = theCat['properties']['name']
    #print(thisCatName)
    #thisCatArea = theCat['properties']['areaInSquareMeters']
    
    GlobalCatchList.append(thisCatName)
    
    #print("Processing " + thisCatName + " with area of " + str(thisCatArea))
        
    fNodeID = currentLink['properties']['from_node']
    
    #print("Node info: " + fNodeID)
    
    fNode = the_network['features'].find_by_id(fNodeID)
    #print(fNode)
    
    fNodeName = fNode[0]['properties']['name']
    #print(fNodeName)
    
    ## recurse
    #print(the_network.upstream_links(fNode[0]))
    for upLink in the_network.upstream_links(fNode[0]):
        #tNodeID = upLink['properties']['to_node']
        #tNode = the_network['features'].find_by_id(tNodeID)
        #print("Now processing: " + upLink['properties']['name'] + " upstream of " + tNode[0]['properties']['name'] + " with initial Region: " + currentReg)
        processLinks(theNetwork, upLink)

    return

In [5]:
#elementMapper = r'P:\projects\eReefsOperational\CSIRO_Locs\WetTropicsElementMapperAll.csv'
#outputLoc = r'P:\projects\eReefsOperational\CSIRO_Locs\WTAreas'

#elementMapper = r'P:\projects\eReefsOperational\CSIRO_Locs\MackayWhitsundayElementMapperAll.csv'
#outputLoc = r'P:\projects\eReefsOperational\CSIRO_Locs\MWAreas'

#elementMapper = r'P:\projects\eReefsOperational\CSIRO_Locs\FitzroyElementMapperAll.csv'
#outputLoc = r'P:\projects\eReefsOperational\CSIRO_Locs\FIAreas'

#elementMapper = r'P:\projects\eReefsOperational\CSIRO_Locs\BurnettMaryElementMapperAll.csv'
#outputLoc = r'P:\projects\eReefsOperational\CSIRO_Locs\BMAreas'

elementMapper = r'P:\projects\eReefsOperational\CSIRO_Locs\CapeYorkElementMapperAll.csv'
outputLoc = r'P:\projects\eReefsOperational\CSIRO_Locs\CYAreas'

if not os.path.exists(outputLoc):
    os.makedirs(outputLoc)
    print("Made dir: " + outputLoc)

elementMapperDF = pd.read_csv(elementMapper)

for index, row in elementMapperDF.iterrows():
    
    GlobalCatchList = []
    
    elementOfInterestName = row['Network Element']
    elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)
    
    outputCSV = os.path.join(outputLoc,elementOfInterestName + ".csv")
    
    if elementOfInterest[0]['properties']['feature_type'] == 'link':
        #Need to add the link/catchent to our collection
        theLinkID = elementOfInterest[0]['id']
        theCat = the_network['features'].find_by_link(theLinkID)[0]
        thisCatName = theCat['properties']['name']
        GlobalCatchList.append(thisCatName)
        print("Added our starting link first")
    
    #print("NOF: " + nodeOfInterest[0]['properties']['name'])
    ## now we begin processing
    for upLink in the_network.upstream_links(elementOfInterest[0]):
        #print("Recursing for link: " + upLink['properties']['name'])
        processLinks(the_network, upLink)

    contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
    #contCatchsDF
    contCatchsDF.to_csv(outputCSV, index=False)
    print("Done exporting " + elementOfInterestName)

    
    
print("All complete")

Made dir: P:\projects\eReefsOperational\CSIRO_Locs\CYAreas
Done exporting Outlet Node106
Done exporting Outlet Node105
Done exporting Outlet Node104
Done exporting Outlet Node101
Done exporting Outlet Node99
Done exporting Outlet Node97
Done exporting Outlet Node96
Done exporting Outlet Node95
Done exporting Outlet Node89
Done exporting Outlet Node93
Done exporting Outlet Node94
Done exporting Outlet Node79
Done exporting Outlet Node77
Done exporting Outlet Node76
Done exporting Outlet Node73
Done exporting Outlet Node71
Done exporting Outlet Node70
Done exporting Outlet Node16
Done exporting Outlet Node69
Done exporting Outlet Node67
Done exporting Outlet Node66
Done exporting Outlet Node64
Done exporting Outlet Node62
Done exporting Outlet Node57
Done exporting Outlet Node56
Done exporting Outlet Node55
Done exporting Outlet Node2
All complete


In [6]:
GlobalCatchList = []

elementOfInterestName = 'link for catchment SC #1643'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_DonRiver.csv'



elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Added our starting link first
Done exporting


In [7]:
GlobalCatchList = []

elementOfInterestName = 'link for catchment SC #1655'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_EuriCreek.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Added our starting link first
Done exporting


In [8]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node12'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_CrocodileCreek.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [9]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node11'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_AlligatorCreek.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [10]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node9'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_RossRiver.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [11]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node8'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_BohleRiver.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [12]:
GlobalCatchList = []

elementOfInterestName = '117002a'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_BlackRiver.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [13]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node6'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_AlthousCreek.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [14]:
GlobalCatchList = []

elementOfInterestName = '117003a'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_BluewaterCreek.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [15]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node14'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_BarramundiCreek.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [16]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node28'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_ElliotRiver.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [17]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node27'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_CapeCreekr.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [18]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node26'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_SandyCreek.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [19]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node21'
outputCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BUAreas\contribCatchs_BurdekinRiver.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [14]:
GlobalCatchList = []

elementOfInterestName = 'link for catchment SC #325'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_UpperBurnett.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Added our starting link first
Done exporting


In [15]:
GlobalCatchList = []

elementOfInterestName = 'link for catchment SC #107'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_BarkerBarambah.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Added our starting link first
Done exporting


In [16]:
GlobalCatchList = []

elementOfInterestName = 'link for catchment SC #244'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_AuburnBoyne1.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Added our starting link first
Done exporting


In [17]:
GlobalCatchList = []

elementOfInterestName = 'link for catchment SC #178'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_AuburnBoyne2.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Added our starting link first
Done exporting


In [18]:
GlobalCatchList = []

elementOfInterestName = 'link for catchment SC #501'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_UpperMary.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Added our starting link first
Done exporting


In [19]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node18'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_NorthBurrum.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [20]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node19'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_SouthBurrum.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [21]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node16'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_SouthElliott.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [22]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node14'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_NorthElliott.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [23]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node20'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_HerveyBay.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting
